<a href="https://colab.research.google.com/github/JR-26/Classification-problems/blob/main/Cat%20vs%20Dog%20Classifier/Cat_vs_Dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

 99% 1.05G/1.06G [00:12<00:00, 206MB/s]
100% 1.06G/1.06G [00:12<00:00, 94.7MB/s]


In [ ]:
import zipfile
jr=zipfile.ZipFile('/content/dogs-vs-cats.zip')
jr.extractall('/content')
jr.close()

In [ ]:
jr='/content/dogs_vs_cats/test'

In [ ]:
d = {'cat': 0, 'dog': 1, 'cats': 0, 'dogs': 1, 'other': 2}
def load_data():
    images = []
    labels = []

    size = (64, 64)

    print('loading from data file', end="")

    for folder in os.listdir(jr):
        fol = folder.strip('._')
        path = os.path.join(jr, fol)
        print(fol, end='|')

        for image_name in os.listdir(path):
            try:
                temp_img = cv2.imread(os.path.join(path, image_name))
                temp_img = cv2.resize(temp_img, size, interpolation=cv2.INTER_AREA)
                images.append(temp_img)
                labels.append(d[fol])
                temp_img = cv2.flip(temp_img, flipCode=1)
                images.append(temp_img)
                labels.append(d[fol])
            except Exception as e:
                print(f"Error processing image {image_name}: {str(e)}")

    if not images or not labels:
        print("Error: Images or labels list is empty.")
        return None, None, None, None

    images = np.array(images)
    images = images.astype('float64') / 255.0
    labels = np.array(labels)
    labels = keras.utils.to_categorical(labels, num_classes=2)

    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

    print("x_train shape:", x_train.shape)
    print("x_test shape:", x_test.shape)
    print("y_train shape:", y_train.shape)
    print("y_test shape:", y_test.shape)

    return x_train, x_test, y_train, y_test

# Replace the existing load_data function with the modified one
x_train, x_test, y_train, y_test = load_data()
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

i = Input(shape=x_train[0].shape)
x = Conv2D(32, (3, 3), activation="relu", padding='same')(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), activation="relu", padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(2, 2)(x)

x = Conv2D(64, (3, 3), activation="relu", padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation="relu", padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(2, 2)(x)

x = Conv2D(128, (3, 3), activation="relu", padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation="relu", padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(2, 2)(x)

x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
num_classes = 2
x = Dense(num_classes, activation='softmax')(x)

model = Model(i, x)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

batch_size = 32
steps_per_epoch = len(x_train) // batch_size if len(x_train) % batch_size == 0 else len(x_train) // batch_size + 1

r = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
              validation_data=(x_test, y_test),
              steps_per_epoch=steps_per_epoch,
              epochs=15)

loading from data filedogs|cats|Images shape: (10000, 64, 64, 3)
Labels shape: (10000, 2)
x_train shape: (8000, 64, 64, 3)
x_test shape: (2000, 64, 64, 3)
y_train shape: (8000, 2)
y_test shape: (2000, 2)
Epoch 1/15
250/250 [==============================] - 291s 1s/step - loss: 1.0425 - accuracy: 0.6136 - val_loss: 0.7497 - val_accuracy: 0.5035
Epoch 2/15
250/250 [==============================] - 288s 1s/step - loss: 0.6070 - accuracy: 0.6737 - val_loss: 0.5828 - val_accuracy: 0.7130
Epoch 3/15
250/250 [==============================] - 290s 1s/step - loss: 0.6029 - accuracy: 0.6773 - val_loss: 0.6066 - val_accuracy: 0.6635
Epoch 4/15
250/250 [==============================] - 301s 1s/step - loss: 0.5684 - accuracy: 0.7048 - val_loss: 0.6192 - val_accuracy: 0.6670
Epoch 5/15
250/250 [==============================] - 276s 1s/step - loss: 0.5555 - accuracy: 0.7197 - val_loss: 0.5679 - val_accuracy: 0.7095
Epoch 6/15
250/250 [==============================] - 281s 1s/step - loss: 0.5080